# Table of Contents
 <p><div class="lev1"><a href="#Summary"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></div><div class="lev1"><a href="#Imports"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></div><div class="lev1"><a href="#Download"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download</a></div><div class="lev1"><a href="#Load-data"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load data</a></div><div class="lev2"><a href="#PDB-chain-/-mutation-(DF1)"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>PDB chain / mutation (DF1)</a></div><div class="lev2"><a href="#UniProt-info-(DF2)"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>UniProt info (DF2)</a></div><div class="lev2"><a href="#Pfam-clan-(DF3)"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Pfam clan (DF3)</a></div><div class="lev2"><a href="#Partner-chain-(DF4)"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Partner chain (DF4)</a></div><div class="lev2"><a href="#Summary"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Summary</a></div><div class="lev1"><a href="#Save-to-database"><span class="toc-item-num">5&nbsp;&nbsp;</span>Save to database</a></div>

# Summary


- `partner_pdb_chain` is null if there are multiple mutations affecting both chains. This may have to be adjusted in ELASPIC...

----

# Imports

In [1]:
%run imports.ipynb

2016-08-21 20:22:41.089770


In [2]:
%run mysqld.ipynb

MySQL database already running...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2016-08-21 20:22:41.251897


In [3]:
NOTEBOOK_NAME = 'ab_bind'
os.environ['NOTEBOOK_NAME'] = NOTEBOOK_NAME
os.makedirs(NOTEBOOK_NAME, exist_ok=True)

# Download

In [4]:
INPUT_URL = (
    "http://onlinelibrary.wiley.com/store/10.1002/pro.2829/asset/supinfo/"
    "pro2829-sup-0002-suppinfo02.xlsx?v=1&s=0778cd0f4d79eacebd5011408d1c605963a8fe4b"
)
INPUT_FILE = op.join(NOTEBOOK_NAME, 'pro2829-sup-0002-suppinfo02.xlsx')

In [5]:
%%bash -s $INPUT_URL $INPUT_FILE
set -ev
INPUT_URL="$1"
INPUT_FILE="$2"

if [[ ! -e "$INPUT_FILE" ]] ; then
    wget -c --no-verbose "$INPUT_URL" -O "$INPUT_FILE"
    # git lfs track "./$INPUT_FILE"
    git add -f "./$INPUT_FILE" 
fi

INPUT_URL="$1"
INPUT_FILE="$2"

if [[ ! -e "$INPUT_FILE" ]] ; then
    wget -c --no-verbose "$INPUT_URL" -O "$INPUT_FILE"
    # git lfs track "./$INPUT_FILE"
    git add -f "./$INPUT_FILE" 
fi


# Load data

In [6]:
DF = pd.read_excel(op.join(NOTEBOOK_NAME, 'pro2829-sup-0002-suppinfo02.xlsx'))

In [7]:
DF.head(5)

,#PDB,Partners(A_B),Protein-1,Protein-2,Mutation,ddG(kcal/mol),PDB Res. (Angstroms),PDB R-value,PDB R-free,PDB pH,PDB T (K),PDB MolProbity clashscore,PDB DOI,Assay Name,Assay pH,Assay Temp (Celcius)
0,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),"D:L483T,D:V486P,D:H487A,D:A488M,D:I491L,D:A492...",0.57,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN
1,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A488G,2.49,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN
2,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A488V,0.60,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN
3,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A492G,0.41,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN
4,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A492V,0.19,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN


## PDB chain / mutation (DF1)

In [8]:
def get_mutation_props(partners, mutations):
    assert len(partners.split('_')) == 2
    # pdb_mutation
    pdb_mutation = mutations.replace(',', '.').replace(':', '_')
    # validate
    mut_chains = [m.split('_')[0] for m in pdb_mutation.split('.')]
    mut_residues = [m.split('_')[1] for m in pdb_mutation.split('.')]
    if len(set(mut_chains)) > 1:
        print("Mutations '{}' affect multiple chains! Skipping...".format(mutations))
        return np.nan, np.nan, np.nan
    # pdb_chain
    pdb_chain = pdb_mutation[0]
    # partner_chain_id
    partner_pdb_chains = [c for c in partners.split('_') if pdb_chain not in c]
    assert len(partner_pdb_chains) == 1
    partner_pdb_chain = partner_pdb_chains[0]
    return pdb_chain, partner_pdb_chain, pdb_mutation


assert get_mutation_props('AB_CD', 'B:N56A') == ('B', 'CD', 'B_N56A')
assert get_mutation_props('AB_CD', 'D:N55A') == ('D', 'AB', 'D_N55A')
assert get_mutation_props('AB_CD', 'B:N56A,B:N55A') == ('B', 'CD', 'B_N56A.B_N55A')
assert get_mutation_props('AB_CD', 'B:N56A,D:N55A') == (np.nan, np.nan, np.nan)

Mutations 'B:N56A,D:N55A' affect multiple chains! Skipping...


In [9]:
DF['pdb_id'] = DF['#PDB'].apply(lambda x: x.split('_')[-1].lower())

DF['pdb_chain'], DF['partner_pdb_chain'], DF['pdb_mutation'] = list(zip(*(
    get_mutation_props(*x)
    for x in DF[['Partners(A_B)', 'Mutation']].values
)))

DF['ddg_exp'] = DF['ddG(kcal/mol)']

Mutations 'H:D32A,C:K97A' affect multiple chains! Skipping...
Mutations 'H:W98A,C:K97A' affect multiple chains! Skipping...
Mutations 'H:W98A,C:S100A' affect multiple chains! Skipping...
Mutations 'H:W98A,C:Y20A' affect multiple chains! Skipping...
Mutations 'H:Y53A,C:D101A' affect multiple chains! Skipping...
Mutations 'H:Y53A,C:L75A' affect multiple chains! Skipping...
Mutations 'H:Y53A,C:W62A' affect multiple chains! Skipping...
Mutations 'H:Y53A,C:W63A' affect multiple chains! Skipping...
Mutations 'L:N32A,C:K96A' affect multiple chains! Skipping...
Mutations 'L:S91A,C:R21A' affect multiple chains! Skipping...
Mutations 'L:S91A,C:Y20A' affect multiple chains! Skipping...
Mutations 'L:Y96A,C:R21A' affect multiple chains! Skipping...
Mutations 'L:Y96A,C:S100A' affect multiple chains! Skipping...
Mutations 'A:Y32A,D:R106A' affect multiple chains! Skipping...
Mutations 'A:Y49A,D:H33A' affect multiple chains! Skipping...
Mutations 'A:Y49A,D:N55A' affect multiple chains! Skipping...
Muta

In [10]:
DF.head()

,#PDB,Partners(A_B),Protein-1,Protein-2,Mutation,ddG(kcal/mol),PDB Res. (Angstroms),PDB R-value,PDB R-free,PDB pH,PDB T (K),PDB MolProbity clashscore,PDB DOI,Assay Name,Assay pH,Assay Temp (Celcius),pdb_id,pdb_chain,partner_pdb_chain,pdb_mutation,ddg_exp
0,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),"D:L483T,D:V486P,D:H487A,D:A488M,D:I491L,D:A492...",0.57,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_L483T.D_V486P.D_H487A.D_A488M.D_I491L.D_A492...,0.57
1,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A488G,2.49,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A488G,2.49
2,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A488V,0.60,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A488V,0.60
3,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A492G,0.41,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A492G,0.41
4,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A492V,0.19,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A492V,0.19


In [11]:
DF1_bak = DF.copy()

## UniProt info (DF2)

In [12]:
DF = DF1_bak.copy()

In [13]:
pdb_ids = sorted(set(DF['pdb_id']))
print(pdb_ids)
print(len(pdb_ids))

['1ak4', '1bj1', '1cz8', '1dqj', '1dvf', '1ffw', '1jrh', '1jtg', '1ktz', '1mhp', '1mlc', '1n8z', '1t83', '1vfb', '1yy9', '2jel', '2ny7', '2nyy', '2nz9', '3bdy', '3be1', '3bn9', '3hfm', '3k2m', '3ngb', '3nps', '3wjj']
27


In [14]:
logger.setLevel(logging.DEBUG)

In [15]:
len('GVALICMFWPDESTYQNKRH')

20

In [16]:
sifts_dfs = {
    pdb_id: ascommon.pdb_tools.sifts.get_sifts_data(pdb_id)
    for pdb_id in pdb_ids
}

In [17]:
sifts_dfs['3wjj'].head()

,comments,is_observed,pdb_aa,pdb_chain,pdb_id,pfam_id,resnum,uniprot_aa,uniprot_id,uniprot_position,residx
0,Not_Observed,False,E,A,3wjj,NaN,216,E,P01857,99,1
1,Not_Observed,False,P,A,3wjj,NaN,217,P,P01857,100,2
2,Not_Observed,False,K,A,3wjj,NaN,218,K,P01857,101,3
3,Not_Observed,False,S,A,3wjj,NaN,219,S,P01857,102,4
4,"Engineered mutation,Not_Observed",False,S,A,3wjj,NaN,220,C,P01857,103,5


In [18]:
assert not any(df is None for df in sifts_dfs.values())

In [19]:
reload(kmtools)
reload(kmtools.pdb_tools.sifts)

<module 'kmtools.pdb_tools.sifts' from '/home/kimlab1/strokach/working/kmtools/kmtools/pdb_tools/sifts.py'>

In [20]:
# Add info to core
def get_sifts_data(pdb_id, pdb_chain, pdb_mutation):
    sifts_df = sifts_dfs[pdb_id]
    try:
        result = kmtools.pdb_tools.sifts.convert_pdb_mutations_to_uniprot(
            pdb_id, pdb_chain, pdb_mutation, sifts_df)
    except kmtools.pdb_tools.sifts.SIFTSError as e:
        logger.error(e)
        result = {}
    return result

results = [get_sifts_data(*x) for x in DF[['pdb_id', 'pdb_chain', 'pdb_mutation']].dropna().values]
results_df = pd.DataFrame(results)

SIFTS failed to match residue (1dqj, H, D, 32, None, 0)
SIFTS failed to match residue (1dqj, H, W, 98, None, 0)
SIFTS failed to match residue (1dqj, H, Y, 33, None, 0)
SIFTS failed to match residue (1dqj, H, Y, 50, None, 0)
SIFTS failed to match residue (1dqj, H, Y, 53, None, 0)
SIFTS failed to match residue (1dqj, L, N, 31, None, 0)
SIFTS failed to match residue (1dqj, L, N, 32, None, 0)
SIFTS failed to match residue (1dqj, L, S, 91, None, 0)
SIFTS failed to match residue (1dqj, L, Y, 50, None, 0)
SIFTS failed to match residue (1dqj, L, Y, 96, None, 0)
PDB AA and UniProt AA are not the same! (1dvf, A, Y, 50, None, 0) (Y != N (P01635))
No mutation mapping available! (1dvf, B_D100A, B, D100A):
    [{'pdb_chain': 'B', 'uniprot_aa': nan, 'pfam_id': nan, 'uniprot_pos': nan, 'uniprot_id': nan}]
No mutation mapping available! (1dvf, B_E98A, B, E98A):
    [{'pdb_chain': 'B', 'uniprot_aa': nan, 'pfam_id': nan, 'uniprot_pos': nan, 'uniprot_id': nan}]
PDB AA and UniProt AA are not the same! (1dv

In [21]:
_before = DF.shape[0]
DF = DF.join(
    results_df.rename(
        columns=lambda x: x.partition('_sifts')[0].replace('mutations', 'mutation')))
assert _before == DF.shape[0]

In [22]:
DF.head(5)

,#PDB,Partners(A_B),Protein-1,Protein-2,Mutation,ddG(kcal/mol),PDB Res. (Angstroms),PDB R-value,PDB R-free,PDB pH,PDB T (K),PDB MolProbity clashscore,PDB DOI,Assay Name,Assay pH,Assay Temp (Celcius),pdb_id,pdb_chain,partner_pdb_chain,pdb_mutation,ddg_exp,pfam_id,uniprot_id,uniprot_mutation
0,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),"D:L483T,D:V486P,D:H487A,D:A488M,D:I491L,D:A492...",0.57,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_L483T.D_V486P.D_H487A.D_A488M.D_I491L.D_A492...,0.57,NaN,P12497,L215T.V218P.H219A.A220M.I223L.A224P.M228I
1,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A488G,2.49,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A488G,2.49,NaN,P12497,A220G
2,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A488V,0.60,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A488V,0.60,NaN,P12497,A220V
3,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A492G,0.41,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A492G,0.41,NaN,P12497,A224G
4,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A492V,0.19,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A492V,0.19,NaN,P12497,A224V


In [23]:
DF2_bak = DF.copy()

## Pfam clan (DF3)

In [24]:
DF = DF2_bak.copy()

In [25]:
pfam_a_clans = (
    pd.read_sql_table('pfam_a_clans', db_remote.engine, schema='pfam')
)

In [26]:
pfam_a_clans.head()

,pfam_id,clan_id,clan_name,pfam_name,pfam_description
0,PF00389,CL0325,Form_Glyc_dh,2-Hacid_dh,"D-isomer specific 2-hydroxyacid dehydrogenase,..."
1,PF00198,CL0149,CoA-acyltrans,2-oxoacid_dh,2-oxoacid dehydrogenases acyltransferase (cata...
2,PF04029,None,None,2-ph_phosp,2-phosphosulpholactate phosphatase
3,PF03171,CL0029,Cupin,2OG-FeII_Oxy,2OG-Fe(II) oxygenase superfamily
4,PF01073,CL0063,NADP_Rossmann,3Beta_HSD,3-beta hydroxysteroid dehydrogenase/isomerase ...


In [27]:
DF['pfam_clan'] = DF['pfam_id'].map(pfam_a_clans.set_index('pfam_id')['clan_id'])

In [28]:
DF['pfam_clan'].notnull().sum()

187

In [29]:
DF3_bak = DF.copy()

## Partner chain (DF4)

In [30]:
DF = DF3_bak.copy()

In [31]:
def get_partner_uniprot(partner_chian, sifts_df):
    sifts_df = sifts_df[sifts_df['pdb_chain'] == partner_chian]
    partner_uniprot_ids = sifts_df['uniprot_id'].dropna().drop_duplicates().tolist()
    if len(partner_uniprot_ids) == 0:
        return np.nan
    elif len(partner_uniprot_ids) == 1:
        return partner_uniprot_ids[0]
    else:
        raise Exception(partner_uniprot_ids)

In [32]:
sifts_dfs['1ak4'].head()

,comments,is_observed,pdb_aa,pdb_chain,pdb_id,pfam_id,resnum,uniprot_aa,uniprot_id,uniprot_position,residx
0,"T,loop",True,M,A,1ak4,NaN,1,M,P62937,1,1
1,"T,loop",True,V,A,1ak4,NaN,2,V,P62937,2,2
2,"T,loop",True,N,A,1ak4,NaN,3,N,P62937,3,3
3,"T,loop",True,P,A,1ak4,NaN,4,P,P62937,4,4
4,"E,strand",True,T,A,1ak4,NaN,5,T,P62937,5,5


In [33]:
get_partner_uniprot('D', sifts_dfs['1ak4'])

'P12497'

In [34]:
DF['partner_uniprot_id'] = [
    get_partner_uniprot(partner_chain, sifts_dfs[pdb_id])
    for pdb_id, partner_chain
    in DF[['pdb_id', 'partner_pdb_chain']].values
]

In [35]:
DF4_bak = DF.copy()

## Summary

In [36]:
print2("Number of rows:", DF.shape[0])
print('-' * 80)

print2("Number of missing uniprots:", DF['uniprot_id'].isnull().sum())
print2("Number of missing mutations:", DF['uniprot_mutation'].isnull().sum())
print2("Number of missing uniprots mutations:", 
       DF[['uniprot_id', 'uniprot_mutation']].isnull().any(axis=1).sum())
print('-' * 80)

print2("Number of missing partner uniprots:", DF['partner_uniprot_id'].isnull().sum())
print2("Number of missing partner uniprot mutations:", 
       DF[['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation']].isnull().any(axis=1).sum())
print('-' * 80)

print2("Number of missing pfams:", DF['pfam_id'].isnull().sum())

Number of rows:                                             1101
--------------------------------------------------------------------------------
Number of missing uniprots:                                 782
Number of missing mutations:                                782
Number of missing uniprots mutations:                       782
--------------------------------------------------------------------------------
Number of missing partner uniprots:                         646
Number of missing partner uniprot mutations:                1014
--------------------------------------------------------------------------------
Number of missing pfams:                                    831


# Save to database

In [37]:
DF = DF4_bak.copy()

In [38]:
DF.head()

,#PDB,Partners(A_B),Protein-1,Protein-2,Mutation,ddG(kcal/mol),PDB Res. (Angstroms),PDB R-value,PDB R-free,PDB pH,PDB T (K),PDB MolProbity clashscore,PDB DOI,Assay Name,Assay pH,Assay Temp (Celcius),pdb_id,pdb_chain,partner_pdb_chain,pdb_mutation,ddg_exp,pfam_id,uniprot_id,uniprot_mutation,pfam_clan,partner_uniprot_id
0,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),"D:L483T,D:V486P,D:H487A,D:A488M,D:I491L,D:A492...",0.57,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_L483T.D_V486P.D_H487A.D_A488M.D_I491L.D_A492...,0.57,NaN,P12497,L215T.V218P.H219A.A220M.I223L.A224P.M228I,NaN,P62937
1,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A488G,2.49,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A488G,2.49,NaN,P12497,A220G,NaN,P62937
2,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A488V,0.60,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A488V,0.60,NaN,P12497,A220V,NaN,P62937
3,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A492G,0.41,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A492G,0.41,NaN,P12497,A224G,NaN,P62937
4,1AK4,A_D,huCyc-A,HIV-1 CAPSID (N-Term),D:A492V,0.19,2.36,0.238,0.306,7,100,4.71,10.1016/S0092-8674(00)81823-1,SPR,6.5,NaN,1ak4,D,A,D_A492V,0.19,NaN,P12497,A224V,NaN,P62937


In [39]:
columns = [
    'uniprot_id', 'partner_uniprot_id', 'uniprot_mutation',
    'pdb_id', 'pdb_chain', 'partner_pdb_chain', 'pdb_mutation',
    
]

In [40]:
DF[columns].head()

,uniprot_id,partner_uniprot_id,uniprot_mutation,pdb_id,pdb_chain,partner_pdb_chain,pdb_mutation
0,P12497,P62937,L215T.V218P.H219A.A220M.I223L.A224P.M228I,1ak4,D,A,D_L483T.D_V486P.D_H487A.D_A488M.D_I491L.D_A492...
1,P12497,P62937,A220G,1ak4,D,A,D_A488G
2,P12497,P62937,A220V,1ak4,D,A,D_A488V
3,P12497,P62937,A224G,1ak4,D,A,D_A492G
4,P12497,P62937,A224V,1ak4,D,A,D_A492V


In [41]:
DF[DF['partner_pdb_chain'].isnull()].shape

(376, 26)

In [42]:
groupby_columns = [
    'pdb_id', 'pdb_chain', 'partner_pdb_chain', 'pdb_mutation',
]
extra_columns = [
    'uniprot_id', 'partner_uniprot_id', 'uniprot_mutation', 'pfam_id', 'pfam_clan',
]
data_columns = [
    'ddg_exp'
]

In [43]:
# Average over duplicate mutations
df = (
    DF
    .groupby(groupby_columns)
    .agg({**{c: lambda x: x.iloc[0] for c in extra_columns}, **{c: 'mean' for c in data_columns}})
    .reset_index()
)

In [44]:
print(DF.shape[0])
print(df.shape[0])

1101
671


In [45]:
t = db.import_df(
    df[groupby_columns + extra_columns + data_columns], 
    NOTEBOOK_NAME)

Loading data into MySQL table: 'ab_bind'...
mysql --local-infile --socket=/tmp/strokach/protein_interaction_energy/mysql.sock -u root protein_interaction_energy -e "load data local infile '/tmp/strokach/protein_interaction_energy/ab_bind.tsv' into table `ab_bind` fields terminated by '\t' optionally enclosed by '\"' ignore 1 lines;  show warnings;" 
Running locally
Command ran successfully:



In [46]:
t.name

'ab_bind'

In [47]:
t.create_indexes([
    (['pdb_id', 'pdb_chain', 'partner_pdb_chain', 'pdb_mutation'], True),
    (['uniprot_id', 'partner_uniprot_id', 'uniprot_mutation'], False),
])

In [48]:
t.add_idx_column()

671

In [56]:
!ls -lh /home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.*

-rw-rw---- 1 strokach kimlab 2.9K Aug 21 20:16 /home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.frm
-rw-rw---- 1 strokach kimlab  19K Aug 21 20:16 /home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYD
-rw-rw---- 1 strokach kimlab  31K Aug 21 20:16 /home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYI


In [50]:
t.compress()

system_command: 'myisampack --no-defaults '/home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYI''
Remember to run myisamchk -rq on compressed tables
system_command: 'myisamchk -rq '/home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYI''
- check record delete-chain
- recovering (with sort) MyISAM-table '/home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYI'
Data records: 671
- Fixing index 1
- Fixing index 2
- Fixing index 3
File size before: 0.04MB
File size after: 0.02MB
File size savings: 0.02MB (0.48%)


(CompletedProcess(args=['myisampack', '--no-defaults', '/home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYI'], returncode=0, stdout='Remember to run myisamchk -rq on compressed tables\n', stderr=''),
 CompletedProcess(args=['myisamchk', '-rq', '/home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYI'], returncode=0, stdout="- check record delete-chain\n- recovering (with sort) MyISAM-table '/home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYI'\nData records: 671\n- Fixing index 1\n- Fixing index 2\n- Fixing index 3\n", stderr=''))

In [57]:
!ls -lh /home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.*

-rw-rw---- 1 strokach kimlab 2.9K Aug 21 20:16 /home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.frm
-rw-rw---- 1 strokach kimlab  19K Aug 21 20:16 /home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYD
-rw-rw---- 1 strokach kimlab  31K Aug 21 20:16 /home/kimlab1/database_data/biodb/recipes/protein_interaction_energy/notebooks/mysqld/protein_interaction_energy/ab_bind.MYI


In [53]:
print(datetime.datetime.now())

2016-08-21 20:23:10.896972
